# 1 - Sequence to Sequence Learning with Neural Nertworks
- pytorch 와 torchtext를 활용해서 한 시퀀스에서 다른 시퀀스로 이동하는 머신러닝 모델을 구축할 것입니다.<br>
- 한글에서 영어로 번역할때 수행되지만 모델은 한 시퀀스에서 다른 시퀀스로 이동하는 것과 관련된 모든 문제에 적용될 수 있습니다. 즉, 동일한 언어의 시퀀스에서 더 짧은 시퀀스로 이동합니다.<br> 
- 첫 번째 노트북에서는 Neural Networks 논문의 모델을 구현하여 일반 개념을 이해하기 쉽게 할 것입니다.

## Introduction
- 가장 일반적인 시퀀스 대 시퀀스(seq2seq) 모델은 인코더-디코더 모델로 일반적으로 순환 신경망(RNN)을 사용하여 소스(입력) 문장을 단일 벡터로 인코딩합니다.
- 이 노트북에서는 이 단일 벡터를 context벡터라고 부를 것입니다. 컨텍스트 벡터는 전체 입력 문장의 추상적 표현으로 생각할 수 있습니다. 그런 다음 이 벡터는 한 번에 한 단어씩 생성하여 대상(출력)문장을 출력하는 방법을 학습하는 두 번째 RNN에 의해 디코딩됩니다.

<p align="center"><img src="https://github.com/bentrevett/pytorch-seq2seq/raw/49df8404d938a6edbf729876405558cc2c2b3013//assets/seq2seq1.png"></p>

- 위 이미지는 translation예시를 보여줍니다. 입력/소스 문장 "guten morgen"은 임베딩 레이어(노란색)를 통과한 다음 인코더(녹색)에 입력됩니다. 또한 시퀀스 시작(`<sos>`)및 시퀀스 끝(`<eos>`)토큰을 문장의 시작과 끝에 각각 추가합니다.
- 각 시간 단계에서 인코더RNN에 대한 입력은 현재 단어 e(xt)의 임베딩 e와 이전 시간 단계 ht-1 및 인코더 RNN의 숨겨진 상태입니다. 새로운 은닉 상태 ht를 출력합니다. 지금까지는 은닉 상태를 문장의 백터표현으로 생각할 수 있습니다. RNN은 다음 두 가지의 함수로 나타낼 수 있습니다. $e(x_{t})$ 및 $h_{t-1}$:
$$ h_{t} = EncoderRNN(e(x_{t}),h_{t-1})$$
여기서는 일반적으로 RNN이라는 용어를 사용합니다. LSTM또는 GRU와 같은 모든 반복 아키텍처가 될 수 있습니다.
우리는 $ X=(x_{1},x_{2},...,x_{T})$, 여기서 $x_{1}=<sos>,x_{2}=guten$ 등 초기 은닉 상태, $h_{0}$ 는 일반적으로 0 또는 학습된 매개변수로 초기화됩니다.
마지막 단어, $x_{T}$ 는 임베딩 레이어를 통해 RNN으로 전달되었으며 최종 은닉 상태를 사용합니다. $h_{T}$,컨텍스트 벡터, 즉 $h_{T}=z$ 이것은 전체 소스 문장의 벡터 표현입니다.
이제 컨텍스트 벡터 $z$ 가 있으므로 출력/목표 문장 "좋은 아침"을 얻기 위해 디코딩을 시작할 수 있습니다. 다시 말하지만, 시퀀스 토큰의 시작과 끝을 대상 문장에 추가합니다. 각 시간 단계에서 디코더 RNN(파란색)에 대한 입력은 이전 시간 단계의 숨겨진 상태뿐만 아니라 현재 단어 현재 단어 ($y_{t}$)의 임베딩, d 및 이전 시간 단계 $s_{t-1}$ 의 숨겨진 상태이며, 여기서 초기 디코더 숨김 상태 $s_{0}$ 은 컨텍스트 벡터, 즉 초기 디코더 숨김 상태 $s_{0}=z$ 이다. 따라서 인코더와 유사하게 우리는 디코더를 다음과 같이 나타낼 수 있다.
$$ s_{t}=DecoderRNN(d(y_{t}),s_{t-1}) $$
입력/소스 임베딩 레이어 $e$ 와 출력/타겟 임베딩 레이어 $d$ 는 모두 다이어그램에서 노란색으로 표시되지만 고유한 매개변수가 있는 두 개의 서로 다른 임베딩 레이어입니다.

디코더에서 우리는 숨겨진 상태에서 실제 단어로 이동해야 하므로 각 시간 단계에서 $s_{t}$ 는(보라색으로 표시된 선형 레이어를 통과함으로써) 시퀀스의 다음 단어라고 생각하는 것을 예측합니다.
$$\hat y_{t} = f(s_{t})$$
디코더의 단어는 항상 시간 단계당 하나씩 차례로 생성됩니다. 디코더에 대한 첫 번째 입력인 $y_{1}$ 에 대해 항상 `<sos>` 를 사용하지만 후속 입력인 $y_{t>1} $ 에 대해서는 시퀀스의 실제 정답인 다음 단어인 $y_{t}$ 를 사용하고 때떄로 디코더에 의해 예측된 단어를 사용합니다. $y_{t}$ 를 사용하고 떄떄로 디코더에 의해 예측된 단어를 사용합니다.
$\hat y_{t-1}$ 이것을 교사 강제라고 합니다 자세한 내용은 [여기](https://machinelearningmastery.com/teacher-forcing-for-recurrent-neural-networks/)를 참고하세요.
우리의 모델을 훈련/테스트할 때 우리는 항상 목표 문장에 얼마나 많은 단어가 있는지 알고 잇으므로 일단 많이 맞으면 단어 생성을 멈춥니다. 추론하는 동안 모델이 `<eos>`토큰을 출력할 떄까지 또는 특정 양의 단어가 생성된 후 단어를 계속 생성하는 것이 일반적입니다.
목표 문장을 예측했다면, $\hat Y = \lbrace \hat y_{1}, \hat y_{2},...,\hat y_{r} \rbrace$, 실제 목표 문장과 비교합니다. $Y = \lbrace y_{1},y_{2},...,y_{T} \rbrace$, 손실을 계산합니다. 그런 다음 이 손실을 사용하여 모델의 모든 매개변수를 업데이트합니다.

## Preparing Data

### 필요 라이브러리 import

In [61]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd

from torchtext.legacy.datasets import Multi30k
from torchtext.legacy.data import Field, BucketIterator

import spacy
import numpy as np

import random
import math
import time

from konlpy.tag import Mecab
mecab = Mecab(dicpath=r"C:\mecab\mecab-ko-dic")

- 고정된 결과를 위해서 임의의 시드를 고정합니다.

In [62]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

### 데이터 불러오기

In [63]:
df = pd.read_excel("data\kor_eng_corpus.xlsx")

In [64]:
train_df = df[:100000]
test_df = df[100000:]

In [65]:
train_df.to_csv("train_data.csv", index=False)
test_df.to_csv("test_data.csv", index=False)

- 다음으로 토크나이저를 생성합니다. 토크나이저는 문장을 포함하는 문자열을 해당 문자열을 구성하는 개별 토큰 목록으로 바꾸는 데 사용됩니다.
- 우리는 지금부터 문장이 단어의 연속이라고 말하는 대신 토큰의 연속이라는 것에 대해 이야기 할것입니다. 차이점은 "good" 과  "morning"은 단어이자 토큰이지만 "!"은 단어가 아니라 토큰입니다.
- spacy는 각 모델의 토크나이저에 엑세스할수 있도록 로드해야 하는 각 언어에 대한 모델이 있습니다. 여기서는 영어모델을 활용할 것입니다.
- 한글 형태소 분석은 mecab활용 할 것이며, 데이터셋은 [ai_hub](https://aihub.or.kr/)의 한영 병력코퍼스를 활용했습니다.

In [67]:
spacy_en = spacy.load('en_core_web_sm')

In [68]:
def tokenize_en(text):
  """
  Tokenizes English text from a string into a list of string (tokens)
  """
  return [tok.text for tok in spacy_en.tokenizer(text)]

### 한영 번역이므로 SRC:한글 -> TRG:영어로 설정

In [73]:
#* 데이터셋에 있는 컬럼을 모두 지정해주어야 splits할때 올바르게 적용이된다.

ID = Field(sequential =False,
          use_vocab=False)

SRC = Field(sequential=True,
            init_token='<SOS>',
            eos_token='<eos>',
            use_vocab=True,
            tokenize=mecab.morphs,
            lower=True,
            batch_first=True,
            fix_length=20
)

TRG = Field(sequential=True,
            use_vocab=True,
            tokenize=tokenize_en,
            lower=True,
            init_token='<sos>',
            eos_token='<eos>',
            batch_first=True,
            fix_length=20
)

### Field 하이퍼파라미터
- sequential : 시퀀스 데이터 여부(True가 기본값)
- use_vocab : 단어 집합을 만들 것인지 여부 (True가 기본값)
- tokenize : 어떤 토큰화 함수를 사용할 것인지 지정.(string.split이 기본값)
- lower : 영어 데이터를 전부 소문자화한다. (False가 기본값)
- batch_first : 미니 배치 자원을 맨 앞으로 하여 데이터를 불러올 것인지 여부 (False가 기본값)
- is_target : 레이블 데이터 여부 (False가 기본값)
- fix_length : 최대 허용 길이.이 길이에 맞춰서 패딩(padding)진행

In [74]:
# torchtext 0.9.9 버전에맞는 경로설정
from torchtext.legacy.data import TabularDataset

train_data, test_data = TabularDataset.splits(
  path=".",train='train_data.csv', test='test_data.csv', format="csv",
  fields=[('ID',ID),('KOR',SRC),('ENG',TRG)], skip_header=True
)

In [77]:
print(vars(train_data[2]))

{'ID': '3', 'KOR': ['푸', '리토', '의', '베스트셀러', '는', '해외', '에서', '입소문', '만', '으로', '4', '차', '완', '판', '을', '기록', '하', '였', '다', '.'], 'ENG': ['purito', "'s", 'bestseller', ',', 'which', 'recorded', '4th', 'rough', '-cuts', 'by', 'words', 'of', 'mouth', 'from', 'abroad', '.']}


In [72]:
# 100000개씩 훈련, 테스트 데이터셋 구분
print(f"Number of training examples: {len(train_data.examples)}")
#print(f"Number of validation examples: {len(valid_data.examples)}")
print(f"Number of testing examples: {len(test_data.examples)}")

Number of training examples: 100000
Number of testing examples: 100000


- 다음으로 한글과 영어의 VOCAB을 만듭니다. VOCAB은 인덱스 번호로 각 각 고유 토큰과 연결됩니다. 한글과 영어의 VOCAB은 별개입니다.

In [78]:
SRC.build_vocab(train_data, min_freq = 2)
TRG.build_vocab(train_data, min_freq = 2)

In [84]:
print(f"Unique tokens in source (de) vocabulary: {len(SRC.vocab)}")
print(f"Unique tokens in target (en) vocabulary: {len(TRG.vocab)}")


Unique tokens in source (de) vocabulary: 17870
Unique tokens in target (en) vocabulary: 14676


데이터 준비의 마지막 단계는 iterators를 생성하는 것입니다. 이것은 src속성(숫자화된 소스 문장의 배치를 포함하는 pytorch텐서)과 trg속성(숫자화된 소스 문장의 배치를 포함하는 pytorch텐서)이 있는 데이터 배치를 반환하기 위해 반복될 수 있습니다. 숫자화는 어휘를 사용하여 읽을 수 있는 토큰 시퀀스에서 해당 인덱스 시퀀스로 변환되었다고 말하는 좋은 방법입니다.

또한  torch.device를 정의해야 합니다. 이것은 텐서를 GPU에 배치할것인지 여부를 torchtext에 전달합니다. 우리는 컴퓨터에서 GPU가 감지되면 TURE를 반환하는 torch.cuda.is_available()함수를 사용합니다. 이 device를 iterator에 전달합니다.

반복자를 사용하여 예제 배치를 얻을 때 모든 소스 문장이 대상 문장과 동일한 길이로 채워져 있는지 확인해야 합니다. 운좋게도, torchtext iterator가 우리를 위해 이것을 처리합니다

표준 iterator 대신 bucketiterator를 사용합니다. 소스 문장과 대상 문장 모두에서 패딩의 양을 최소화하는 방식으로 일괄 처리를 생성하기 때문입니다.


In [85]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [88]:
BATCH_SIZE = 128

train_iterator, test_iterator = BucketIterator.splits(
    (train_data, test_data),
    batch_size = BATCH_SIZE, 
    device = device)

# Seq2Seq model 설계
세 부분으로 나눠서 모델을 제작할 예정입니다. 인코더, 디코더 및 인코더와 디코더를 캡슐화하는 seq2seq 모델은 각각과 인터페이스하는 방법을 제공한다.

### Encoder
먼저 인코더인 2계층 LSTM입니다. 우리가 구현하고 있는 논문은 4레이어 LSTM을 사용하지만 훈련 시간을 위해 이것을 2레이어로 줄였습니다. 다중 계층 RNN의 개념은 2계층에서 4계층으로 쉽게 확장할 수 있습니다.

다층 RNN의 경우 입력 문장 X는 삽입된 후 RNN의 첫 번째(하위) 계층과 숨겨진 상태로 이동합니다. $ H = {h_{1},h_{2},...,h_{t}} $ 이 계층의 출력은 위 계층의 RNN에 대한 입력으로 사용됩니다. 따라서 각 레이어를 위 첨자로 표현하면 첫 번째 레이어의 숨겨진 상태가 다음과 같이 표시됩니다.
$$ h_{t}^1 = EncoderRNN^1(e(x_{t}),h_{t-1}^1) $$
두 번째 레이어의 숨겨진 상태는 다음과 같이 제공됩니다.
$$ h^2_{t} = EncoderRNN^2(h_{t}^1,h^2_{t-1}) $$
다중 레이어 RNN을 사용하면 레이어,$h_{0}^l$ 마다 입력으로 초기 은닉 상태도필요합니다. 그리고 우리는 레이어,$z^l$ 마다 컨텍스트 벡터도 출력합니다.

LSTM에 대해 더 자세한 설명은 [여기](https://colah.github.io/posts/2015-08-Understanding-LSTMs/)를 참고하세요.LSTM이 단지 숨김 상태를 가져다가 시간단계별로 새로운 숨김 상태를 반환하는 것이 아니라, 셀 상태도 시간단계별로 가져와서 반환하는 RNN의 한 유형이라는 것입니다.
$$ h_{t} = RNN(e(x_{t}),h_{t-1}) $$
$$ (h_{t},c_{t}) = LSTM(e(x_{t}),h_{t-1},c_{t-1}) $$
우리는 $ c_{t} $ 는 다른유형의 숨겨진 상태라고 생각할수 있습니다. h1과 유사하게 c1은 모두 제로텐서로 초기화 될 것이다. 또한, 우리의 컨텍스트 벡터는 이제 최종 숨겨진 상태와 최종 셀 상태, 즉 둘 다일 것이다. i.e. $ z^l = (h_{T}^l,c_{T}^l) $
멀티레이어 방정식을 LSTM으로 확장하면 다음과 같습니다.
$$ (h_{t}^1,c_{t}^1) = EncoderLSTM^1(e(x_{t}),(h_{t-1}^l, c_{t-1}^1)) $$
$$ (h_{t}^2,c_{t}^2) = EncoderLSTM^2(h_{t}^1,(h_{t-1}^2,c_{t-1}^2)) $$
첫 번째 레이어의 숨겨진 상태만 두 번째 레이어에 입력으로 전달되고 셀 상태는 전달되지 않습니다.
따라서 인코더는 다음과 같이 보입니다:
<p align="center"><img src="https://github.com/bentrevett/pytorch-seq2seq/raw/49df8404d938a6edbf729876405558cc2c2b3013//assets/seq2seq2.png"></p>
Seq2Seq 모델은 인코더, 디코더 및 디바이스(있는경우 GPU에 텐서를 배치하는데 사용됨)를 사용합니다.
이 구현을 위해 우리는 `인코더`와 `디코더`에서 레이어의 수와 숨겨진(셀)차원이 동일한지 확인해야 합니다. 이것은 항상 그런 것은 아니며 SEQUENCE-TO-SEQUENCE 모델에서 동일한 수의 레이어나 동일한 은닉 차원 크기가 반드시 필요한 것은 아닙니다.
그러나 다른 수의 레이어를 갖는 것과 같은 작업을 수행할 경우 이를 처리하는 방법에 대한 결정을 내려야 합니다. 예를 들어 인코더에 2개의 레이어가 있고 디코더에 1개의 레이어만 있는 경우 이를 어떻게 처리합니까? 디코더가 출력한 두 컨텍스트 벡터의 평균을 구합니까? 둘 다 선형 레이어를 통과합니까? 최상위 레이어의 컨텍스트 벡터만 사용합니까? 등.

우리가 나아갈 방향은 SRC문장, TRG문장 및 티처포싱 비율을 사용합니다. 티처포싱 비율은 우리의 모델을 훈련할떄 주로 사용됩니다. 디코딩할 때 각 시간단계에서 대상 시퀀스의 다음 토큰이 디코딩된 이전 토큰 $\hat y_{t+1} = f(s_{t}^L) $ 에서 무엇인지 예측합니다. 티처포싱과 동일한 확률로 시퀀스의 실제 ground_truth 다음 토큰을 다음 시간 단계동안 디코더에 대한 입력으로 사용합니다.그러나 가능성 1 - 티처포싱 비율 에서는 시퀀스의 실제 다음 토큰과 일치하지 않더라도 모델이 모델에 대한 다음 입력으로 예측한 토큰을 사용합니다.

우리가 사용할 방법에서 가장 먼저 하는 일은 모든 예측을 저장할 출력 텐서를 만드는 것입니다. $ \hat Y $

그런 다음 입력/소스 문장인 SRC를 인코더에 입력하고 최종 숨겨진 셀 상태를 수신합니다.

디코더에 대한 첫 번째 입력은 시퀀스 시작(`<sos>`) 토큰입니다.TRG텐서에는 이미 `<SOS>`토큰이 추가되어 있으므로(TRG필드에서 init_token을 정의할 때까지 거슬러 올라감) $ y_{1} $ 로 슬라이싱 합니다. 우리는 목표 문장의 길이(max_len)을 알고 있으므로 여러 번 반복합니다. 디코더에 입력된 마지막 토큰은 `<eos>`토큰 이전의 토큰입니다. `<eos>`토큰은 디코더에 입력되지 않습니다.

루프를 반복할 떄마다 다음을 수행합니다:
- input전달, 이전 hidden과 이전 셀 상태 $ (y_{t},s_{t-1},c_{t-1}) $ 를 디코더로 전달
- 예측을 받고, 다음 hidden상태와 다음 셀상태 $ (\hat y_{t+1},s_{t},c_{t}) $ 디코더로부터
- 우리의 예측, $ \hat y_{t+1}/output $ 을 우리의 예측 텐서, $ \hat Y/outputs $ 에 놓는다.
- 우리가 티처포싱을 할것인지 여부를 결정합니다.
  - 만약에 티처포싱을 한다면 다음 입력은 시퀀스의 ground-truth 다음 토큰입니다. $ y_{t+1}/trg[t] $
  - 그렇지 않은 경우 다음 입력은 시퀀스에서 예측된 다음 토큰입니다. $ \hat y_{t+1}/top1 $, 출력텐서에 대해 argmax를 수행하여 얻습니다.

모든 예측이 끝나면 예측으로 가득 찬 텐서를 반환합니다. $ \hat Y/outputs $

**Note:**디코더 루프는 0이 아닌 1에서 시작합니다. 이것은 출력 텐서의 0번째 요소가 모두 0으로 유지된다는 것을 의미합니다. 따라서 TRG 및 출력은 다음과 같습니다.
$$ trg=[<sos>,y_{1},y_{2},y_{3},<eos>] $$
$$ outputs = [0,\hat y_{1},\hat y_{2}, \hat y_{3}, <eos>] $$
나중에 손실을 계산할 때 각 텐서의 첫 번째 요소를 잘라 다음을 얻습니다.
$$ trg = [y_{1},y_{2},y_{3},<eos>] $$
$$ outputs = [\hat y_{1},\hat y_{2},\hat y_{3}, <eos>] $$

In [ ]:
class Encoder(nn.Module):
  def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
    super().__init__()

    self.hid_dim = hid_dim
    self.n_layers = n_layers